In [1]:
import os
os.chdir('/home/data/')

from time import time

In [2]:
import sqlite3
import pandas as pd

from sqlalchemy import create_engine

## Create an empty sqlite database file

In [3]:
!rm tutorial.db

In [4]:
con = sqlite3.connect("tutorial.db")

- alternatively, use

```python
con = create_engine('sqlite:///tutorial.db')
```

## Create a _cursor_ object to interact with it

In [5]:
cur = con.cursor()

---
# Loading Data into the Database

- This step might take a while if your CSV file is larger than a few GBs, 

- But the benefits outweigh the wait time;
    - you can use `pd.read_sql` tools to pull data from the database without worrying about memory constraints.
    - you can use tools like `Metabase` or any SQL editor to write aggregations and reductions on big data locally.  
    
    
- [Note] Avoid using `SELECT *` as it will load all data into memory. 

- Use `WHERE` statements and the `LIMIT` clause each time.

### Load KDD Data

In [7]:
kdd_chunks = pd.read_csv(
    "kdd.csv", 
    chunksize=10**6, 
    error_bad_lines=False
)

In [8]:
t0 = time()
for chunk in kdd_chunks:
    """
    Fill the table by reading a large text file in chunks.
    Each chunk is just a pandas DataFrame
    Filter/transform the data as needed here.
    """
    chunk.to_sql(
        name='kdd', 
        con=con, 
        if_exists='append',
        index=False
    )
    
print("Loading into db finished in {} seconds.".format(time()-t0))

Loading into db finished in 74.79250454902649 seconds.


### Load Flights Data

In [9]:
flights_chunks = pd.read_csv(
    'flights.csv', 
    chunksize=10**6,
    error_bad_lines=False
)

In [10]:
t0 = time()

for chunk in flights_chunks:
    """
    Fill the table by reading a large text file in chunks.
    Each chunk is just a pandas DataFrame
    Filter/transform the data as needed here.
    """
    chunk.to_sql(
        name='flightDelays', 
        con=con, 
        if_exists='append',
        index=False
    )
    
print("Loading into db finished in {} seconds.".format(time()-t0))

Loading into db finished in 93.90438771247864 seconds.


## Load NYC Taxi Data

- Get it [here](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)

In [7]:
url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-{}.csv"

In [11]:
[url.format(str(i).zfill(2)) for i in range(1, 13)]

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-01.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-02.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-03.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-04.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-05.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-06.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-07.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-08.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-09.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-10.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-11.csv',
 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-12.csv']

---
## Check if loading went well

In [13]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('kdd',), ('flightDelays',)]

---
## Run SQL queries

In [6]:
pd.read_sql(
    """
    SELECT Origin,Dest,count(*) as Flights
    FROM flightDelays 
    GROUP BY 1,2
    ORDER BY 3 DESC
    LIMIT 10
    """, 
    con=con)

,Origin,Dest,Flights
0,OGG,HNL,16099
1,HNL,OGG,15876
2,LAX,LAS,14385
3,LAS,LAX,13815
4,HNL,LIH,13156
5,LIH,HNL,13030
6,SAN,LAX,12779
7,LAX,SAN,12767
8,BOS,LGA,12263
9,LAS,PHX,12228


In [16]:
pd.read_sql("SELECT count(*) FROM kdd", con=con)

,count(*)
0,4898431
